# Projet Otto 
Malik Kazi Aoual & Nouredine Nour
M2 AIC

Ce notebook résume les différentes étapes de notre cheminement tout au long de ce projet. Il suit le plan du rapport. 

## Imports

In [1]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfTransformer

# I - Donnés

Todo : 
1. Histogramme de représentation des classes dans les données
2. Mise en évidence des cas extrèmes df.describe

In [2]:
df_raw = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test_raw = pd.read_csv("../data/test.csv").drop("id",axis=1)

# II - Résultats préliminaires

Todo: 
1. Multinomial Naive Bayes avec et sans normalisation

In [ ]:
# Chargement des donnés 
df_raw = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test_raw = pd.read_csv("../data/test.csv").drop("id",axis=1)
# Récupération des labels
labels = df_raw["target"].values
y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)
# Récupérations des features sous forme de ndarray
df_raw = df_raw.drop("target", axis=1)
X = df_raw.values
# X_test = df_test_raw.values
print('X shape : '+str(X.shape))
print('y shape : '+str(y.shape))

In [ ]:
# Split pour validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)

target_names = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

In [ ]:
clf = KNeighborsClassifier(n_neighbors=10, n_jobs=-1)
%time clf.fit(X_train, y_train)
print("Fit terminé")
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Log loss (Kaggle metric) : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [ ]:

# NE PAS LANCER (sauf si vous avez tout votre temps...)
clf = SVC(probability=True)
%time clf.fit(X_train, y_train)
print("Fit terminé")
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Log loss (Kaggle metric) : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

### Multinomial Naive Bayes sans normalisation

In [ ]:
clf = MultinomialNB()
%time clf.fit(X_train, y_train)
print("Fit terminé")
pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Log loss (Kaggle metric) : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

### Multinomial Naive Bayes avec normalisation 


In [ ]:
clf = MultinomialNB()
X_train_norm = Normalizer().fit_transform(X_train)
X_test_norm = Normalizer().fit_transform(X_test)
%time clf.fit(X_train_norm, y_train)
print("Fit terminé")
pred = clf.predict_proba(X_test_norm)
pred_for_acc = clf.predict(X_test_norm)
print("Log loss (Kaggle metric) : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [ ]:
rf = ExtraTreesClassifier(n_estimators=250, n_jobs=-1)
X_train_norm = Normalizer().fit_transform(X_train)
X_test_norm = Normalizer().fit_transform(X_test)
%time clf.fit(X_train, y_train)
print("Fit terminé")
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Log loss (Kaggle metric) : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [ ]:
clf = KNeighborsClassifier(n_neighbors=10, n_jobs=-1)
%time clf.fit(X_train_norm, y_train)
print("Fit terminé")
%time pred = clf.predict_proba(X_test_norm)
pred_for_acc = clf.predict(X_test_norm)
print("Log loss (Kaggle metric) : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))


# III - Arbres et calibration

Todo :
1. lancer la meilleur repres et eventuellement la tester en cv

In [ ]:
clf_tmp = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1)
clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
%time clf.fit(X_train, y_train)
print("Fit terminé")
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

# IV - Feature selection et Nouvelle représentation

Todo:
1. Relancer les algos précédents avec %time pour voir 
2. SVM
3. KNN eventuellement CV pour le K


## Séléction des features

Afin de pouvoir lancer les classifieurs tels que KNN ou SVM trop lents sur la représentation initiale, nous avons voulu tenter de réduire le nombre de features pour alléger la représentation des données.

Todo :
1. récupérer les données Selction chi2
2. récupérer les données RMVE
3. tracer en fonction du nombre de features gardés les perf de différent classifieur avec %time

In [ ]:
# Feature selection sur MNB

X_norm = Normalizer().fit_transform(X)
k_bests=[5, 9, 20 ,30, 50, 80, 93]

mnb = MultinomialNB()
clf = mnb
results_mnb = []
for k_best in k_bests:
    selection = SelectKBest(chi2, k=k_best)
    X_new = selection.fit_transform(X_norm, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2,
                                                      random_state=42)
 
    print('MNB for k_best = '+str(k_best))
    %time clf.fit(X_train, y_train)
    %time pred = clf.predict_proba(X_test)
    results_mnb.append(log_loss(y_test, pred, eps=1e-15, normalize=True))
    print("--- Log loss (Kaggle metric) : %.2f ---" % log_loss(y_test, pred, eps=1e-15, normalize=True))
        
        

In [ ]:
# Feature selection sur KNN k = 10

X_norm = Normalizer().fit_transform(X)
k_bests=[5, 9, 20 ,30, 50, 80, 93]
knn = KNeighborsClassifier(n_neighbors=10, n_jobs=-1)
clf = knn
results_knn = []
for k_best in k_bests:
    selection = SelectKBest(chi2, k=k_best)
    X_new = selection.fit_transform(X_norm, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2,
                                                      random_state=42)
 
    print('KNN for k_best = '+str(k_best))
    %time clf.fit(X_train, y_train)
    %time pred = clf.predict_proba(X_test)
    results_knn.append(log_loss(y_test, pred, eps=1e-15, normalize=True))
    print("--- Log loss (Kaggle metric) : %.2f ---" % log_loss(y_test, pred, eps=1e-15, normalize=True))
        
        

In [ ]:
# Feature selection sur KNN k = 10

X_norm = Normalizer().fit_transform(X)
k_bests=[9]
svm = SVC(C=100, gamma=0.001)
clf = svm
results_knn = []
for k_best in k_bests:
    selection = SelectKBest(chi2, k=k_best)
    X_new = selection.fit_transform(X_norm, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2,
                                                      random_state=42)
 
    print('SVC for k_best = '+str(k_best))
    %time clf.fit(X_train, y_train)
    %time pred = clf.predict_proba(X_test)
    results_knn.append(log_loss(y_test, pred, eps=1e-15, normalize=True))
    print("--- Log loss (Kaggle metric) : %.2f ---" % log_loss(y_test, pred, eps=1e-15, normalize=True))
        
        

In [ ]:
# Feature selection sur RF 100

X_norm = Normalizer().fit_transform(X)
k_bests=[5, 9, 20 ,30, 50, 80, 93]

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
clf = rf
results_rf = []
for k_best in k_bests:
    selection = SelectKBest(chi2, k=k_best)
    X_new = selection.fit_transform(X_norm, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2,
                                                      random_state=42)
 
    print('RF for k_best = '+str(k_best))
    %time clf.fit(X_train, y_train)
    %time pred = clf.predict_proba(X_test)
    results_rf.append(log_loss(y_test, pred, eps=1e-15, normalize=True))
    print("--- Log loss (Kaggle metric) : %.2f ---" % log_loss(y_test, pred, eps=1e-15, normalize=True))
        
        

In [ ]:
%matplotlib inline
plt.plot(k_bests, results_mnb, color='r', label='Multinomial Naive Bayes')
plt.plot(k_bests, results_rf, color='black', label='Random Forest')
plt.plot(k_bests, results_knn, color='b', label='KNN')
plt.xlabel('Nombre de features gardées dans la représentation')
plt.ylabel('Log loss')
plt.legend()
plt.plot
plt.savefig('plot.jpg', format='jpg', dpi=800)

In [ ]:
plt.savefig('plot.jpg', format='jpg', dpi=800)

In [ ]:
results_knn = np.array(results_knn)
results_mnb = np.array(results_mnb)
results_rf = np.array(results_rf)
print(results_mnb.min())
print(results_knn.min())
print(results_rf.min())

### Nouvelle représentation Euclidean

In [9]:
df = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test = pd.read_csv("../data/test.csv").drop("id",axis=1)

labels = df["target"].values
print(type(labels))
print(labels.shape)

df_train = df.drop("target", axis=1)

X = df_train.values
#X_test = df_test.values

# print(X.shape)
# print(X_test.shape)

y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)

unique_labels = np.unique(y)

tf = TfidfTransformer()

print("Construction des centroïdes")

X_tf = tf.fit_transform(X)

centroids = []
for label in unique_labels:
    same_labels = X_tf[np.where(y == label)]
    centroids.append(same_labels.sum(axis = 0))
centroids = np.asarray(centroids).reshape(9,93)
print(centroids.shape)
print("Construction terminée")
X_dc = pairwise_distances(X_tf, centroids, metric='cosine')
#X_test_dc = pairwise_distances(X_test, centroids, metric='minkowski')

X_train, X_test, y_train, y_test = train_test_split(X_dc, y, test_size=0.2, random_state=42)

print("X_train shape : ", X_train.shape)
print("X_test shape : ", X_test.shape)


<class 'numpy.ndarray'>
(61878,)
Construction des centroïdes
(9, 93)
Construction terminée
X_train shape :  (49502, 9)
X_test shape :  (12376, 9)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_dc, y, test_size=0.2,
random_state=42)

target_names = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

In [11]:
clf = MultinomialNB()
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))


CPU times: user 27.9 ms, sys: 13.6 ms, total: 41.5 ms
Wall time: 26.4 ms
CPU times: user 25.7 ms, sys: 13.1 ms, total: 38.9 ms
Wall time: 17.6 ms
Kaggle log loss metric : 1.71
Accuracy : 0.47
             precision    recall  f1-score   support

    class_1       0.00      0.00      0.00       399
    class_2       0.50      0.99      0.66      3178
    class_3       0.00      0.00      0.00      1561
    class_4       0.00      0.00      0.00       538
    class_5       0.00      0.00      0.00       565
    class_6       0.45      0.94      0.61      2884
    class_7       0.00      0.00      0.00       552
    class_8       0.00      0.00      0.00      1674
    class_9       0.00      0.00      0.00      1025

avg / total       0.23      0.47      0.31     12376



/Users/nour/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))


Le random forest sur la nouvelle représentation pour 9 features fait 0.81 alors qu'il avait donné 3.26 sur la représentation à 9 features (chi2).

In [ ]:
clf = KNeighborsClassifier(n_neighbors=10, n_jobs=-1)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))


Knn fait 1.86 au lieu de 4.46. De plus le temps de computation est grandement diminué ce qui nous permettera d'eventuellement faire de la cv sur le K.

In [ ]:
clf_tmp = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1)
clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
%time clf.fit(X_train, y_train)
print("Fit terminé")
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [ ]:
clf_tmp = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1)
clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
%time clf.fit(X_train, y_train)
print("Fit terminé")
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [ ]:
clf_2 = SVC(gamma=0.001, C=100, probability=True, cache_size=4000, verbose=True, 
            class_weight='balanced')
%time clf_2.fit(X_train, y_train)
print("Fit terminé")
%time pred_2 = clf_2.predict_proba(X_test)
pred_for_acc_2 = clf_2.predict(X_test)
print("%.2f" % log_loss(y_test, pred_2, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc_2, target_names=target_names))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import cross_val_score

In [ ]:
knn = KNeighborsClassifier()
param_dist = {"n_neighbors": sp_randint(1, 500)}
n_iter_search = 20
random_search = RandomizedSearchCV(knn, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)
random_search.fit(X_dc, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % (2, n_iter_search))
report(random_search.cv_results_)

In [ ]:
neighbors = [5, 10, 15, 20, 30, 40, 100, 200, 300, 400]
cv_scores = []
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_dc, y, cv=5, scoring='log_loss')
    cv_scores.append(scores.mean())

In [ ]:
%matplotlib inline
plt.plot(neighbors, cv_scores)

print(cv_scores)

In [ ]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
report(random_search.cv_results_)

In [ ]:
neighbors = [200, 300, 400, 500, 600, 700, 800, 900, 1000]
cv_scores = []
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_dc, y, cv=5, scoring='log_loss')
    cv_scores.append(scores.mean())

In [ ]:
%matplotlib inline
plt.plot(neighbors, cv_scores)

print(cv_scores)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=18, n_jobs=-1)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))


# V - Mélange
Todo:
Récupérer les différents classifieurs précédent en pickle et tenter des mélanges